In [1]:
""" Performing a large-scale experiment on cifar10 """

import keras
import logging
import numpy as np
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
import keras.regularizers as regularizers
from keras.models import load_model

# Record settings
LOG_FORMAT = "%(levelname)s %(asctime)s - %(message)s"
logging.basicConfig(filename="swish_first_exp_log.txt",format = LOG_FORMAT, level = logging.DEBUG, filemode = "a")
logs = logging.getLogger()

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [7]:
# Swish activation function
# x*sigmoid(x)
def leaky(x):
    return K.maximum(0.3*x, x)

In [8]:
def create(act):
    baseMapNum = 32
    weight_decay = 1e-4

    model = Sequential()
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()
    return model

In [9]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [10]:
def schedule(x):
    if x < 75: 
        return 0.001
    elif x < 100:
        return 0.0005
    elif x < 125:
        return 0.0003
    elif x<150:
        return 0.00015
    elif x<175:
        return 0.000075
    elif x<200: 
        return 0.000035
    elif x<225:
        return 0.000017
    else:
        return 0.000012

In [11]:
# Create model and get ready - reproducible cell
act, act_name = leaky, "leaky"
model = create(act)
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Training - only for 125 eppochs
batch_size = 128
epochs=250
# lr scheduler
lr_1 = keras.callbacks.LearningRateScheduler(schedule)

# Fit the model
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
# Save the model
model.save('models/leaky_relu/leaky_at_2.h5')
# Display logs
print(his.history)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

390/390 [==============================] - 154s 395ms/step - loss: 0.7549 - acc: 0.7730 - val_loss: 0.6329 - val_acc: 0.8161
Epoch 34/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7490 - acc: 0.7738 - val_loss: 0.6141 - val_acc: 0.8216
Epoch 35/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7473 - acc: 0.7754 - val_loss: 0.5810 - val_acc: 0.8386
Epoch 36/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7389 - acc: 0.7775 - val_loss: 0.5655 - val_acc: 0.8410
Epoch 37/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7412 - acc: 0.7783 - val_loss: 0.5700 - val_acc: 0.8421
Epoch 38/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7299 - acc: 0.7807 - val_loss: 0.5777 - val_acc: 0.8365
Epoch 39/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7374 - acc: 0.7797 - val_loss: 0.5976 - val_acc: 0.8302
Epoch 40/250
390/390 [=========================

390/390 [==============================] - 154s 395ms/step - loss: 0.5747 - acc: 0.8312 - val_loss: 0.4679 - val_acc: 0.8708
Epoch 93/250
390/390 [==============================] - 154s 396ms/step - loss: 0.5703 - acc: 0.8350 - val_loss: 0.4547 - val_acc: 0.8726
Epoch 94/250
390/390 [==============================] - 154s 395ms/step - loss: 0.5770 - acc: 0.8316 - val_loss: 0.4657 - val_acc: 0.8716
Epoch 95/250
390/390 [==============================] - 154s 395ms/step - loss: 0.5697 - acc: 0.8338 - val_loss: 0.4771 - val_acc: 0.8665
Epoch 96/250
390/390 [==============================] - 154s 395ms/step - loss: 0.5691 - acc: 0.8330 - val_loss: 0.4679 - val_acc: 0.8702
Epoch 97/250
390/390 [==============================] - 154s 395ms/step - loss: 0.5677 - acc: 0.8340 - val_loss: 0.4673 - val_acc: 0.8727
Epoch 98/250
390/390 [==============================] - 154s 395ms/step - loss: 0.5696 - acc: 0.8352 - val_loss: 0.4712 - val_acc: 0.8705
Epoch 99/250
390/390 [=========================

390/390 [==============================] - 154s 395ms/step - loss: 0.4586 - acc: 0.8655 - val_loss: 0.3817 - val_acc: 0.8969
Epoch 152/250
390/390 [==============================] - 154s 395ms/step - loss: 0.4505 - acc: 0.8687 - val_loss: 0.3820 - val_acc: 0.8969
Epoch 153/250
390/390 [==============================] - 154s 395ms/step - loss: 0.4513 - acc: 0.8684 - val_loss: 0.3825 - val_acc: 0.8967
Epoch 154/250
390/390 [==============================] - 154s 396ms/step - loss: 0.4518 - acc: 0.8685 - val_loss: 0.3760 - val_acc: 0.8980
Epoch 155/250
390/390 [==============================] - 154s 395ms/step - loss: 0.4490 - acc: 0.8696 - val_loss: 0.3800 - val_acc: 0.8960
Epoch 156/250
390/390 [==============================] - 154s 396ms/step - loss: 0.4472 - acc: 0.8694 - val_loss: 0.3762 - val_acc: 0.8971
Epoch 157/250
390/390 [==============================] - 154s 395ms/step - loss: 0.4478 - acc: 0.8706 - val_loss: 0.3755 - val_acc: 0.8989
Epoch 158/250
390/390 [==================

390/390 [==============================] - 154s 396ms/step - loss: 0.4194 - acc: 0.8785 - val_loss: 0.3655 - val_acc: 0.9033
Epoch 211/250
390/390 [==============================] - 154s 395ms/step - loss: 0.4193 - acc: 0.8779 - val_loss: 0.3629 - val_acc: 0.9019
Epoch 212/250
390/390 [==============================] - 154s 396ms/step - loss: 0.4196 - acc: 0.8778 - val_loss: 0.3645 - val_acc: 0.9022
Epoch 213/250
390/390 [==============================] - 154s 396ms/step - loss: 0.4246 - acc: 0.8755 - val_loss: 0.3642 - val_acc: 0.9027
Epoch 214/250
390/390 [==============================] - 154s 395ms/step - loss: 0.4190 - acc: 0.8764 - val_loss: 0.3657 - val_acc: 0.9009
Epoch 215/250
390/390 [==============================] - 154s 396ms/step - loss: 0.4227 - acc: 0.8781 - val_loss: 0.3676 - val_acc: 0.9025
Epoch 216/250
390/390 [==============================] - 154s 396ms/step - loss: 0.4229 - acc: 0.8768 - val_loss: 0.3646 - val_acc: 0.9020
Epoch 217/250
390/390 [==================

In [12]:
# Create model and get ready - reproducible cell
act, act_name = leaky, "leaky"
model = create(act)
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Training - only for 125 eppochs
batch_size = 128
epochs=250
# lr scheduler
lr_1 = keras.callbacks.LearningRateScheduler(schedule)

# Fit the model
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
# Save the model
model.save('models/leaky_relu/leaky_at_3.h5')
# Display logs
print(his.history)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
__________

390/390 [==============================] - 154s 395ms/step - loss: 0.7442 - acc: 0.7758 - val_loss: 0.5823 - val_acc: 0.8345
Epoch 34/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7432 - acc: 0.7764 - val_loss: 0.5637 - val_acc: 0.8405
Epoch 35/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7434 - acc: 0.7775 - val_loss: 0.6304 - val_acc: 0.8189
Epoch 36/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7339 - acc: 0.7809 - val_loss: 0.5708 - val_acc: 0.8409
Epoch 37/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7331 - acc: 0.7807 - val_loss: 0.5853 - val_acc: 0.8323
Epoch 38/250
390/390 [==============================] - 154s 394ms/step - loss: 0.7338 - acc: 0.7801 - val_loss: 0.5765 - val_acc: 0.8345
Epoch 39/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7205 - acc: 0.7846 - val_loss: 0.5989 - val_acc: 0.8280
Epoch 40/250
390/390 [=========================

StopIteration: 

In [ ]:
def accuracy(y_pred, y_test):
    y_hat = np.argmax(y_pred, axis=1)
    y = np.argmax(y_test, axis=1)

    good = np.sum(np.equal(y, y_hat))
    return float(good/len(y_test))

In [ ]:
import keras.regularizers as regularizers
ensembler = 0
for i in range(3):
    if i == 0:
        model = create(swish)
        model.load_weights('models/leky_relu/leaky_at_'+str(i+1)+'.h5')
        model.save('models/leky_relu/leaky_at_'+str(i+1)+'.h5')
    else:
        model = load_model('models/leky_relu/leaky_at_'+str(i+1)+'.h5')
    ensembler += model.predict_proba(x_test)
    
print(accuracy(ensembler, y_test))

In [ ]:
# Swish activation function
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)

# Custom activation function 1
# mix between relu and positive part of swish mirrored across x=1
def e_swish_1(x):
    return K.maximum(0.0, x*(2-K.sigmoid(x)))

# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [ ]:
# Create model and get ready - reproducible cell
act, act_name = e_swish_2, "e_swish_2"
model = create(act)
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Training
batch_size = 128
epochs=250
# lr scheduler
lr_1 = keras.callbacks.LearningRateScheduler(schedule)

# Fit the model
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
# Save the model
model.save('models/e_swish/e_swish_2_at_4.h5')
# Display logs
print(his.history)